In [1]:
import json
import sys
import pandas as pd
import requests
from datetime import *

# 전역 변수
accident_data = 'accident.csv'
location_data = 'location.csv'
APP_KEY = '402a35d1a3216ce892c22768b85ef463'    # 발급받은 키 입력
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

In [2]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    print('%s : success for request [%s]' % (datetime.now(), url))

    return resp.text

In [3]:
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)

    # json request
    try:
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
    except:
        json_name = 'NaN'

    return json_name

In [4]:
def get_address(data):
    address = []

    # 경도, 위도 추출해서 동 주소 반환
    for i in range(len(data)):
        longitude = data['경도'][i]
        latitude = data['위도'][i]
        address.append(reverse_geocode(longitude, latitude))

    return address  # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가

In [5]:
def main():
    # 파일 읽기
    data = pd.read_csv(accident_data, encoding='euc-kr')
    location = pd.read_csv(location_data, encoding='euc-kr')

    address = get_address(data)
    data = preprocess(data, address, location)  # 전처리 함수(생략했음)

    # 최종 파일 저장
    data.to_csv('%s_final.csv' % (accident_data[:-4]), sep=',', encoding='euc-kr')

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'accident.csv'

### 지오코딩

In [8]:
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + '402a35d1a3216ce892c22768b85ef463' }
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

In [34]:
addr_to_lat_lon('전라북도 김제시 서암4길 45 (서암동)')

(126.880863694158, 35.807173413668)